This script is currently written with the assumption that all data is located in the same folder (see sl_example_data)

Import libraries

In [ ]:
import openpyxl as xl
import xlrd
import pandas as pd
from pandas import ExcelWriter
from openpyxl import load_workbook
import csv
import os

Establish variables and make directories (These are the only aspects of the script that may need to change)

In [ ]:
folder_name = 'sl_example_data'               
auditory_folder = 'auditory_folder'
visual_folder = 'visual_folder'
auditory_times = pd.read_excel("EV_timing.xlsx", 'Auditory_Times', index = False)
visual_times = pd.read_excel("EV_timing.xlsx", 'Visual_Times', index = False)
speech_struct_1_2 = 'speech_struct_1_2'
tone_rand_1_2 = 'tone_rand_1_2'             
tone_struct_3_4 = 'tone_struct_3_4'
speech_rand_3_4 = 'speech_rand_3_4'
image_rand_1_2 = 'image_rand_1_2'
letter_struct_1_2 = 'letter_struct_1_2'
image_struct_3_4 = 'image_struct_3_4'
letter_rand_3_4 = 'letter_rand_3_4'



os.mkdir(speech_struct_1_2)
os.mkdir(tone_rand_1_2)
os.mkdir(tone_struct_3_4)
os.mkdir(speech_rand_3_4)
os.mkdir(image_rand_1_2)
os.mkdir(letter_struct_1_2)
os.mkdir(image_struct_3_4)
os.mkdir(letter_rand_3_4)



os.makedirs(auditory_folder) #Creat a new folder to save new auditory excel files
os.makedirs(visual_folder)  #Creat a new folder to save new visual excel files
auditory_files = []
visual_files = []

Run program to extract raw event files, create a list that stores event information in individual files, adds timing information and sorts into files compatible with FSL.

In [ ]:
for i in os.listdir(folder_name): #load every file from the folder
    if i.endswith('auditory_1.csv') or i.endswith('auditory_2.csv') or i.endswith('auditory_3.csv') or i.endswith('auditory_4.csv'):
        auditory_files.append(i)
    elif i.endswith('visual_1.csv') or i.endswith('visual_2.csv') or i.endswith('visual_3.csv') or i.endswith('visual_4.csv'):
        visual_files.append(i)


def save_new_folder(files, new_folder_name):
    for list in files:

        file = open(os.path.join(folder_name, list))  # load csv into the python
        numline = len(file.readlines())

        def read_cell(x, y):  # function that can read the cell
            with open(os.path.join(folder_name, list), 'r') as f:
                reader = csv.reader(f)
                y_count = 0
                for n in reader:
                    if y_count == y:
                        cell = n[x]
                        return cell
                    y_count += 1

        a = []  # creat a list to store R S B
        for i in range(1, numline + 1):
            a.append(read_cell(5, i))
        # cell = sheet.cell(row, 2)
        # for list in cell.value:
        #     a.append(list)

        inRow = []  # create another list to store R and S without repeat
        last = None
        for x in a:
            if last != x:
                inRow.append(x)
            last = x

        inRow = [x for x in inRow if x != 'B']  # Delete all the B's
        inRow = [x for x in inRow if x != None]
        print(inRow)

        # for f in os.listdir(folder_name): #rename the new excel file which is .xlsx file
        f_name, f_ext = os.path.splitext(list)
        new_ext = '.xlsx'
        new_name = '{}{}'.format(f_name, new_ext)

        df = pd.DataFrame(inRow)  # write the result into a new excel
        writer = ExcelWriter(os.path.join(new_folder_name, new_name))  # save every file to new folder
        df.to_excel(writer, 'Sheet1', index=False)
        writer.save()

save_new_folder(auditory_files, auditory_folder)
save_new_folder(visual_files, visual_folder)



def save_time(folder, time):
    for i in os.listdir(folder):
        book = load_workbook(os.path.join(folder, i))
        writer = pd.ExcelWriter(os.path.join(folder, i), engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        time.to_excel(writer, 'Sheet1', index=False, startrow=1, startcol=1)
        writer.save()

save_time(auditory_folder,auditory_times)
save_time(visual_folder,visual_times)





auditory_file_1_2 = []
auditory_file_3_4 = []
visual_file_1_2 = []
visual_file_3_4 = []

for i in os.listdir(auditory_folder): #load every file from the folder
    if i.endswith('auditory_1.xlsx') or i.endswith('auditory_2.xlsx'):
        auditory_file_1_2.append(i)
    elif i.endswith('auditory_3.xlsx') or i.endswith('auditory_4.xlsx'):
        auditory_file_3_4.append(i)

for i in os.listdir(visual_folder): #load every file from the folder
    if i.endswith('visual_1.xlsx') or i.endswith('visual_2.xlsx'):
        visual_file_1_2.append(i)
    elif i.endswith('visual_3.xlsx') or i.endswith('visual_4.xlsx'):
        visual_file_3_4.append(i)


def keep_S(files, folder,folder_name):
    for i in files:
        df = pd.read_excel(os.path.join(folder_name,i),index_col=0)
        df = df.drop(["R"])
        # df = df[:]
        #df = df.drop(df.columns[0],axis=1)
        print(df)
        f_name, f_ext = os.path.splitext(i)
        new_ext = '.txt'
        new_name = '{}{}'.format(f_name, new_ext)
        with open(os.path.join(folder, new_name),'w') as outfile:
            df.to_string(outfile, index=False, header=None)

def keep_R(files, folder,folder_name):
    for i in files:
        df = pd.read_excel(os.path.join(folder_name,i),index_col=0)
        df = df.drop(["S"])
        # df = df[:]
        #df = df.drop(df.columns[0],axis=1)
        print(df)
        f_name, f_ext = os.path.splitext(i)
        new_ext = '.txt'
        new_name = '{}{}'.format(f_name, new_ext)
        with open(os.path.join(folder, new_name),'w') as outfile:
            df.to_string(outfile, index=False, header=None)

keep_S(auditory_file_1_2, speech_struct_1_2,auditory_folder)
keep_R(auditory_file_1_2, tone_rand_1_2,auditory_folder)
keep_S(auditory_file_3_4, tone_struct_3_4,auditory_folder)
keep_R(auditory_file_3_4,speech_rand_3_4,auditory_folder)
keep_S(visual_file_1_2,letter_struct_1_2,visual_folder)
keep_R(visual_file_1_2,image_rand_1_2,visual_folder)
keep_S(visual_file_3_4,image_struct_3_4,visual_folder)
keep_R(visual_file_3_4,letter_rand_3_4,visual_folder)
